## 환경 설정 & 라이브러리

In [ ]:
# 필수 라이브러리 불러오기
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
import os
import pandas as pd
from dotenv import load_dotenv

# .env 파일 로드 (OpenAI API Key 불러오기)
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("✅ OpenAI API Key 로드 성공!")
else:
    print("⚠️ OpenAI API Key가 없습니다. .env 파일을 확인하세요.")


OpenAI API Key 로드 성공!


## 말투 데이터셋 로드

In [ ]:
# ✅ 말투 데이터셋 로드
style_data_path = "../../data/tone_dataset.csv"  # 말투 데이터셋 파일 경로
df = pd.read_csv(style_data_path)

# ✅ 랜덤하게 5개의 예제 선택
style_examples = "\n".join([
    f"- 원본: {row['original']}\n  - 변환: {row['converted']}" 
    for _, row in df.sample(5).iterrows()
])

print("✅ 말투 데이터셋 로드 완료!")
print(style_examples)  # 로드된 데이터 확인


## LangChain 프롬프트 설정

In [ ]:
# ✅ LangChain ChatPromptTemplate 설정
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        """당신은 청소년 고민 상담을 도와주는 AI 챗봇입니다.  
사용자가 편안하게 대화를 나눌 수 있도록, 친구처럼 자연스럽게 말해 주세요.  
응답을 생성할 때, 아래 스타일 예제를 참고해서 대화 스타일을 유지하세요.

💡 **말투 스타일 예제 (Few-shot 데이터셋 기반)**:  
{style_examples}

📌 **사용자 입력**:  
"""
    ),
    HumanMessagePromptTemplate.from_template("{user_input}")
])

print("✅ LangChain 프롬프트 설정 완료!")


## 벡터DB 불러오기

In [ ]:
# ✅ 벡터 DB 로드
vectorstore = FAISS.load_local('../../data/db/faiss', OpenAIEmbeddings(), allow_dangerous_deserialization=True)

# ✅ 대화 기반 검색을 위한 Retriever 설정
retriever = vectorstore.as_retriever()

# ✅ Memory 설정 (대화 이력 저장)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

# ✅ Conversational RAG 설정
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

conversation_rag = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": prompt}  # 프롬프트 반영
)

print("✅ Conversational RAG 설정 완료!")


Conversational RAG 설정 완료!


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_10124\1012020170.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")


## RAG 검색 & 대화 흐름 최적화 

In [ ]:
def get_relevant_rag_response(user_input):
    """RAG 검색 실행 후, 결과가 적절한지 판단"""
    response = conversation_rag.invoke({"question": user_input})
    answer = response["answer"]

    # ✅ 검색 결과가 의미 있는 경우만 반환
    if len(answer) > 5:  # 의미 있는 답변인지 간단히 체크 (길이 기준)
        return answer
    else:
        return None  # 의미 없는 경우 RAG 결과 사용 안 함

print("✅ RAG 검색 함수 설정 완료!")


## 챗봇 실행

In [ ]:
def chat_with_bot():
    while True:
        user_input = input("\n당신: ")

        if user_input.lower() in ["exit", "quit"]:
            print("\n대화를 종료합니다. 다음에 또 만나요.\n")
            break

        # ✅ 항상 RAG 실행
        rag_response = get_relevant_rag_response(user_input)

        # ✅ 최종 프롬프트에 데이터 적용
        final_prompt = prompt.format(
            style_examples=style_examples,  # 말투 데이터셋 반영
            user_input=user_input
        )

        # ✅ LLM 호출 (RAG 응답이 있으면 포함)
        if rag_response:
            final_prompt += f"\n📂 참고할 상담 사례:\n{rag_response}"

        # ✅ LLM이 최종 응답 생성
        final_answer = llm.invoke(final_prompt)

        print("\n챗봇:\n", final_answer)

print("✅ 챗봇 실행 준비 완료! 'chat_with_bot()' 실행하면 대화 시작!")


### RAG 테스트

In [ ]:
import sys

print("💬 고민 상담 챗봇 시작! (종료는 'exit' 입력)")

while True:
    user_input = input("👤 사용자: ")  # 사용자 입력 받기
    print("👤 사용자:", user_input)  # 사용자 입력 즉시 출력
    sys.stdout.flush()  # 버퍼 강제 플러시

    if user_input.lower() == "exit":  # 'exit' 입력 시 종료
        print("🔚 상담 챗봇 종료!")
        break

    # ✅ 메모리에 저장된 대화 기록 로드
    chat_history = memory.load_memory_variables({})["chat_history"]

    # ✅ RAG 기반 응답 (자동으로 대화 기록 포함)
    response = conversation_rag.invoke({"question": user_input})

    # ✅ 🔍 RAG 작동 여부 확인
    if response["source_documents"]:  # 검색된 문서가 있을 경우
        print("\n📂 RAG 검색된 문서를 기반으로 답변 중...")
    else:  # 검색된 문서가 없을 경우
        print("\n⚠️ 벡터DB에 검색된 문서 없음. GPT 모델이 직접 답변을 생성합니다.")

    # ✅ 챗봇 응답 출력
    print("🤖 챗봇:", response["answer"])
    sys.stdout.flush()  # 챗봇 출력 후에도 플러시


💬 고민 상담 챗봇 시작! (종료는는 'exit' 입력)
👤 사용자: 나 너무 힘들어

📂 RAG 검색된 문서를 기반으로 답변 중...
🤖 챗봇: "음, 요즘 많이 힘든 것 같구나. 다른 사람들은 잘 지내는 것 같은데, 너만 힘든 기분이 드는 건 정말 속상할 것 같아. 그럴 때는 정말 외롭고 힘들겠지. 최근에 어떤 일이 있어서 그렇게 느끼게 되었어?"
👤 사용자: 애들이 나만 빼고 밥 먹으러 가.

📂 RAG 검색된 문서를 기반으로 답변 중...
🤖 챗봇: "아, 그런 일이 있었구나. 친구들이 나만 빼고 밥 먹으러 가면 정말 속상하고 혼자 남겨진 기분이 들겠어. 나만 소외된 것 같아서 기분이 많이 무거워질 것 같아. 혹시 최근에 친구들과의 관계에서 어떤 변화가 있었던 것 같아? 아니면 어떤 일로 인해 그런 기분이 들었는지 이야기해 줄 수 있어?"
👤 사용자: 그냥 날 질투하나봐

📂 RAG 검색된 문서를 기반으로 답변 중...
🤖 챗봇: "그런 기분이 드는구나.. 누군가가 질투하는 것 같아서 마음이 불편할 것 같아. 그런 상황은 정말 힘들지. 그 친구와의 관계는 어떤지 궁금해. 혹시 어떤 행동들이 그런 생각을 하게 만들었는지 이야기해줄 수 있어?"
👤 사용자: exit
🔚 상담 챗봇 종료!
